## Clustering Assignment 

## Note Book Created 

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

print('Libraries imported.')

Libraries imported.


## Web Scraping Started 
## Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below:

In [7]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install lxml


Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install requests 

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv

print('BeautifulSoup  & csv imported.')

BeautifulSoup  & csv imported.


In [4]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

#print(soup.prettify())
#print('soup ready')

In [5]:
TABLE=soup.find('table',class_='wikitable sortable')
#print(TABLE)

In [6]:
table_rows = TABLE.find_all('tr')

#table_rows

In [8]:
data1 = []
for row in table_rows:
    data1.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data1, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows
#print(df.head())
print(df.tail())


    PostalCode       Borough          Neighbourhood
284        M8Z     Etobicoke              Mimico NW
285        M8Z     Etobicoke     The Queensway West
286        M8Z     Etobicoke  Royal York South West
287        M8Z     Etobicoke         South of Bloor
288        M9Z  Not assigned           Not assigned


## Data transformed into pandas dataframe

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 288 entries, 1 to 288
Data columns (total 3 columns):
PostalCode       288 non-null object
Borough          288 non-null object
Neighbourhood    288 non-null object
dtypes: object(3)
memory usage: 9.0+ KB


In [10]:
df.shape

(288, 3)

## Dataframe cleaned and notebook annotate

In [11]:
df.drop(df[df['Borough']=='Not assigned'].index, axis=0,inplace=True)

In [12]:
df.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [13]:
df1=df.reset_index()

In [14]:
df1.shape

(211, 4)

In [15]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211 entries, 0 to 210
Data columns (total 4 columns):
index            211 non-null int64
PostalCode       211 non-null object
Borough          211 non-null object
Neighbourhood    211 non-null object
dtypes: int64(1), object(3)
memory usage: 6.7+ KB


## More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [16]:
df2=df1.groupby('PostalCode').agg(lambda x:' , '.join(x))
df2.head()

,Borough,Neighbourhood
PostalCode,,
M1B,"Scarborough , Scarborough","Rouge , Malvern"
M1C,"Scarborough , Scarborough , Scarborough","Highland Creek , Rouge Hill , Port Union"
M1E,"Scarborough , Scarborough , Scarborough","Guildwood , Morningside , West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [17]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103 entries, M1B to M9W
Data columns (total 2 columns):
Borough          103 non-null object
Neighbourhood    103 non-null object
dtypes: object(2)
memory usage: 2.4+ KB


## If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.


In [18]:
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']

df2.head()

,Borough,Neighbourhood
PostalCode,,
M1B,"Scarborough , Scarborough","Rouge , Malvern"
M1C,"Scarborough , Scarborough , Scarborough","Highland Creek , Rouge Hill , Port Union"
M1E,"Scarborough , Scarborough , Scarborough","Guildwood , Morningside , West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [19]:
df3=df2.reset_index()
df3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,"Scarborough , Scarborough","Rouge , Malvern"
1,M1C,"Scarborough , Scarborough , Scarborough","Highland Creek , Rouge Hill , Port Union"
2,M1E,"Scarborough , Scarborough , Scarborough","Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Now we can remove the duplicate boroughts as follows:

In [23]:
df3['Borough']= df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',')
df3.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge , Malvern"
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park , Ionview , Kennedy Park"
7,M1L,Scarborough,"Clairlea , Golden Mile , Oakridge"
8,M1M,Scarborough,"Cliffcrest , Cliffside , Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff , Cliffside West"


## In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [24]:
df3.shape

(103, 3)